In [54]:
from difflib import get_close_matches
import pandas as pd
import numpy as np

In [55]:
import sys
sys.path.insert(0, os.path.abspath('../'))
import os
import pyspark as spark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark.sql.functions as f
from pyspark.sql import DataFrame, Row
from functools import reduce
import functools
from pyspark.sql.types import StructType, StructField, StringType, LongType, ArrayType, IntegerType, MapType
from pyspark.sql.functions import *

In [56]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
conf=SparkConf()
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory", "4g")
conf.set("spark.cores.max", "6")

sc = SparkContext.getOrCreate(conf)

spark = SQLContext(sc)

/home/preacher/anaconda3/envs/deltarun/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [57]:
def open_file(bucket_name, s3_file_name):    
    try:
        # obj = s3_client.get_object(Bucket=bucket_name, Key= s3_file_name)
        # df = pd.read_csv(obj['Body']) # 'Body' is a key word
        df = pd.read_parquet(s3_file_name) # 'Body' is a key word
        df.rename(columns = {'brand':'crawled_name'}, inplace = True)
        return df
    except Exception as err:
        print(err)

In [58]:
file_path = '/home/preacher/Bungee/bnt/data_set/saas_team/brand_dict/part-00001-2e2876d3-2300-4ac3-ac95-a5c33a6a33f3-c000.snappy.parquet'

In [59]:

unclustered_brands = open_file('', file_path)
brand_list = unclustered_brands.crawled_name.tolist()
brand_status = {brand: False for brand in brand_list}

In [60]:
clusters = []
for brand, visited in brand_status.items():
    # if not visited:
        cluster = get_close_matches(brand, brand_list)
        print(cluster)
        for brand in cluster:
            # print(brand_status[brand])
            brand_status[brand] = True
            # print(brand_status[brand])
        clusters.append(cluster)
    # else :
    #     print('Skipped')


['(null)']
['***phoenix Performance Disc Vendor']
['1-TDC']
['2 Hounds Design', 'Petrageous Designs', 'PetRageous Designs']
['21st Century']
['3V Caps']
['4health Grain Free', '4health Grain Free Homestyle', 'Health Guard']
['4health Grain Free Homestyle', '4health Grain Free', '4health Homestyle']
['4health Homestyle', '4health Grain Free Homestyle']
['4x4 North America', 'North American Pet', 'Stubben North America']
['5/A Baker Products, Inc.', 'Dare Products Inc', 'Scott Pet Products Inc.']
['8in1']
['9Lives', 'Liver Bits', 'Loveson']
['A E Cage Company', 'A&E Cage Company', 'Cashel Company']
['A&E Cage Company', 'A E Cage Company', 'Cashel Company']
['A.C. Kerman - Pet Products', 'Bergan Pet Products', 'Crown Pet Products']
['ABO Gear']
['ACE Equestrain', 'Jpc Equestrian', 'Shires Equestrian']
['ADM']
['ADVANTAGE II']
['AE', 'OASE']
['AG-Alchemy']
['AKC Pet', 'RC Pet', 'CC Pet']
['ANTIROBE', 'NUTRO']
['APPLE-DEX']
['ASPEN', 'OASE']
['ATTITUDE']
['Abbott Laboratories', 'Thomas Labo

In [74]:
cluster_string = [ [','.join(cluster)] for cluster in clusters]

In [75]:
cluster_string

[['(null)'],
 ['***phoenix Performance Disc Vendor'],
 ['1-TDC'],
 ['2 Hounds Design,Petrageous Designs,PetRageous Designs'],
 ['21st Century'],
 ['3V Caps'],
 ['4health Grain Free,4health Grain Free Homestyle,Health Guard'],
 ['4health Grain Free Homestyle,4health Grain Free,4health Homestyle'],
 ['4health Homestyle,4health Grain Free Homestyle'],
 ['4x4 North America,North American Pet,Stubben North America'],
 ['5/A Baker Products, Inc.,Dare Products Inc,Scott Pet Products Inc.'],
 ['8in1'],
 ['9Lives,Liver Bits,Loveson'],
 ['A E Cage Company,A&E Cage Company,Cashel Company'],
 ['A&E Cage Company,A E Cage Company,Cashel Company'],
 ['A.C. Kerman - Pet Products,Bergan Pet Products,Crown Pet Products'],
 ['ABO Gear'],
 ['ACE Equestrain,Jpc Equestrian,Shires Equestrian'],
 ['ADM'],
 ['ADVANTAGE II'],
 ['AE,OASE'],
 ['AG-Alchemy'],
 ['AKC Pet,RC Pet,CC Pet'],
 ['ANTIROBE,NUTRO'],
 ['APPLE-DEX'],
 ['ASPEN,OASE'],
 ['ATTITUDE'],
 ['Abbott Laboratories,Thomas Laboratories,St Jon Laboratori

In [76]:
column = ['string_clusters']

In [64]:
type(clusters)

list

In [80]:
df = spark.createDataFrame(cluster_string, column)

In [81]:
df.show(truncate=False)

+------------------------------------------------------------------+
|string_clusters                                                   |
+------------------------------------------------------------------+
|(null)                                                            |
|***phoenix Performance Disc Vendor                                |
|1-TDC                                                             |
|2 Hounds Design,Petrageous Designs,PetRageous Designs             |
|21st Century                                                      |
|3V Caps                                                           |
|4health Grain Free,4health Grain Free Homestyle,Health Guard      |
|4health Grain Free Homestyle,4health Grain Free,4health Homestyle |
|4health Homestyle,4health Grain Free Homestyle                    |
|4x4 North America,North American Pet,Stubben North America        |
|5/A Baker Products, Inc.,Dare Products Inc,Scott Pet Products Inc.|
|8in1                             

In [88]:
df2 = df.select(array_sort(split(col("string_clusters"),",")).alias("brand_cluster")).drop("string_clusters")
df2.printSchema()
df2.show(truncate=False)

root
 |-- brand_cluster: array (nullable = true)
 |    |-- element: string (containsNull = true)

+-----------------------------------------------------------------------+
|brand_cluster                                                          |
+-----------------------------------------------------------------------+
|[(null)]                                                               |
|[***phoenix Performance Disc Vendor]                                   |
|[1-TDC]                                                                |
|[2 Hounds Design, PetRageous Designs, Petrageous Designs]              |
|[21st Century]                                                         |
|[3V Caps]                                                              |
|[4health Grain Free, 4health Grain Free Homestyle, Health Guard]       |
|[4health Grain Free, 4health Grain Free Homestyle, 4health Homestyle]  |
|[4health Grain Free Homestyle, 4health Homestyle]                      |
|[4x4 North Am

In [93]:
df3 = df2.filter(size(df2.brand_cluster)>1).drop_duplicates()

In [94]:
df3.show(truncate=False)

+----------------------------------------------------------------------------------------------------------+
|brand_cluster                                                                                             |
+----------------------------------------------------------------------------------------------------------+
|[Akorn, Aleko, Fleck]                                                                                     |
|[Alenza, Altren]                                                                                          |
|[American Animal Health, Ceva Animal Health, epiQ Animal Health]                                          |
|[Amigo, Amino, Pawmigo]                                                                                   |
|[ Inc., ArthroDynamic Technologies, ArthroDynamic Technologies, Chism Technologies LLC.]                  |
|[Aura Frames, Natural Fresh, Naturally Fresh]                                                             |
|[Baxter, Beefeater

In [91]:
#  add index based on the last greatest index present in brand_cluster
#  join with brand_dict to get all brand info
#  convert the res wrt the database store

2656

1765

+-----------------------------------------------------------------------+
|brand_cluster                                                          |
+-----------------------------------------------------------------------+
|[(null)]                                                               |
|[***phoenix Performance Disc Vendor]                                   |
|[1-TDC]                                                                |
|[2 Hounds Design, PetRageous Designs, Petrageous Designs]              |
|[21st Century]                                                         |
|[3V Caps]                                                              |
|[4health Grain Free, 4health Grain Free Homestyle, Health Guard]       |
|[4health Grain Free, 4health Grain Free Homestyle, 4health Homestyle]  |
|[4health Grain Free Homestyle, 4health Homestyle]                      |
|[4x4 North America, North American Pet, Stubben North America]         |
|[ Inc., 5/A Baker Products, Dare Prod